In [2]:
# https://colab.research.google.com/github/climatechange-ai-tutorials/nlp-policy-analysis/blob/main/paris_prompts.ipynb#scrollTo=9jVjv8etAIiQ
import os
import pandas as pd
import numpy as np
import random

# set seeds for reproducibility
np.random.seed(42)
random.seed(42)

# this will signify to the model the start of the input text
INPUT_MARKER = 'Input Text'

# This dictionary just contains the names of the columns to slightly longer names that are easier to understand.
column_names = {'goal': 'Sustainable Development Goal', 'target': 'Subgoal', 'status': 'Status', 'sector': 'Sector', 'response': 'Climate Response', 'infotype': 'Information Type'}

SPECIAL_TOKEN = "**"

In [ ]:
# for now we will skip the special_token as well as the what the "context" column is.

In [ ]:
def build_single_prompt(row, pred_column, obs_column, test_input_only=False):
    ''' Args:
        row (pd.Series): a row from the dataframe
        obs_column (str): the column that we are observing (e.g. text, context)
        test_input_only (bool):
            - if True, only return the input text
                - This is used for the test example.
            - if False, return the input text and the correct answer
                - This will be for in-context learning
        Returns:
            prompt (str): the second portion of the prompt
    '''
    prompt = f'\n{INPUT_MARKER}: {row[obs_column]}\n'
    if test_input_only:
        return prompt

    value = row[pred_column]
    prompt += f'{pred_column}:{value}\n'
    prompt += '\n'
    return prompt


# sample a random row from the dataframe to use as a test example
random_test_row = df_full.sample(1).iloc[0]
test_input = build_single_prompt(random_test_row, pred_column='goal', obs_column='text', test_input_only=True)

prompt_example += test_input
prompt_example += f'{"goal"}:'
# here we have the full instruction and prompt for the model
print(prompt_example)

In [ ]:
import os

# To use the model, you will need to sign up for an OpenAI Account (http://platform.openai.com/signup) and get an API key pasted below.
os.environ["OPENAI_API_KEY"] = "INSERT_YOUR_KEY_HERE"
openai.api_key = os.getenv("OPENAI_API_KEY")

# so we will use this particular model
MODEL_NAME = "text-davinci-003"

def get_model_response(test_prompt, pay=False):
  ''' test_prompt (str)
        - The full text prompt to send to the API for completion.
      pay (bool):
        - If False, return a dummy response. If True, send the prompt to the model API
        - This simple flag to let us test the function without calling the API which costs money.
      Returns:
        response (dict): the response from the API or a dummy response if Pay is False
  '''
  if not pay:
    # this is a dummy response that always returns the class 3
    response = {'choices':[{'text':'3'}]}
  else:
    response = openai.Completion.create(
      model=MODEL_NAME,
      prompt=test_prompt,
      temperature=0,
      max_tokens=4,
      top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0
    )
  return response


# For now we can just test the function, but you can see what happens by setting pay=True
response = get_model_response(prompt_example, pay=False)
example_pred = response['choices'][0]['text']
print(example_pred)